In [ ]:
import cv2
import mediapipe as mp
import numpy as np

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    a = np.array(a)  # Point A coordinates
    b = np.array(b)  # Point B coordinates (vertex)
    c = np.array(c)  # Point C coordinates
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return angle

In [ ]:
def classify_pose(landmarks, frame):
    """Classify the yoga pose based on the angles of joints and display all angles."""
    label = 'Unknown Pose'
    color = (0, 0, 255)  # Default to red for unknown pose

    if landmarks:
        try:
            # Extract landmarks for the required joints
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Calculate angles
            angle1 = calculate_angle(left_shoulder, left_elbow, left_wrist)
            angle2 = calculate_angle(right_shoulder, right_elbow, right_wrist)
            angle3 = calculate_angle(right_elbow, right_shoulder, right_hip)
            angle4 = calculate_angle(left_elbow, left_shoulder, left_hip)
            angle5 = calculate_angle(right_shoulder, right_hip, right_knee)
            angle6 = calculate_angle(left_shoulder, left_hip, left_knee)
            angle7 = calculate_angle(right_hip, right_knee, right_ankle)
            angle8 = calculate_angle(left_hip, left_knee, left_ankle)

            # Define pose based on the angles
            if angle2 > 155 and angle2 < 195 and angle1 > 155 and angle1 < 195:
                if angle4 > 70 and angle4 < 110 and angle3 > 70 and angle3 < 110:
                    if angle8 > 165 and angle8 < 195 or angle7 > 165 and angle7 < 195:
                        if (angle8 > 80 and angle8 < 120) or (angle7 > 80 and angle7 < 120):
                            label = 'Warrior II Pose'
                            color = (255, 215, 0)  # Gold color for Warrior II Pose
                    if (angle8 > 160 and angle8 < 195) and (angle7 > 160 and angle7 < 195):
                        label = 'T Pose'
                        color = (0, 255, 0)  # Green for T Pose
            if (angle8 > 165 and angle8 < 195) or (angle7 > 165 and angle7 < 195):
                if (angle7 > 20 and angle7 < 45) or (angle8 > 20 and angle8 < 45):
                    label = 'Tree Pose'
                    color = (128, 0, 128)  # Purple for Tree Pose

            # Display all angles on the image
            cv2.putText(frame, f'Angles: {angle1:.2f}, {angle2:.2f}, {angle3:.2f}, {angle4:.2f}, {angle5:.2f}, {angle6:.2f}, {angle7:.2f}, {angle8:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # Display the pose label
            cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error in classify_pose: {str(e)}")

    return frame

In [ ]:
def main():
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame)

            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )
                frame = classify_pose(results.pose_landmarks.landmark, frame)

            cv2.imshow('Yoga Pose Image', frame)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

In [ ]:
#输出角度版本
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    a = np.array(a)  # Point A coordinates
    b = np.array(b)  # Point B coordinates (vertex)
    c = np.array(c)  # Point C coordinates
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return angle

def classify_pose(landmarks, frame):
    """Classify the yoga pose based on the angles of joints and display all angles."""
    label = 'Unknown Pose'
    color = (0, 0, 255)  # Default to red for unknown pose

    if landmarks:
        try:
            # Extract landmarks for the required joints
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Calculate angles
            angle1 = calculate_angle(left_shoulder, left_elbow, left_wrist)
            angle2 = calculate_angle(right_shoulder, right_elbow, right_wrist)
            angle3 = calculate_angle(right_elbow, right_shoulder, right_hip)
            angle4 = calculate_angle(left_elbow, left_shoulder, left_hip)
            angle5 = calculate_angle(right_shoulder, right_hip, right_knee)
            angle6 = calculate_angle(left_shoulder, left_hip, left_knee)
            angle7 = calculate_angle(right_hip, right_knee, right_ankle)
            angle8 = calculate_angle(left_hip, left_knee, left_ankle)

            # Define pose based on the angles
            if (angle2 > 160 and angle2 < 200) and (angle1 > 160 and angle1 < 200):  # angle1 = angle2 = 180 degree
                if (angle4 > 70 and angle4 < 110) and (angle3 > 70 and angle3 < 110): # angle3 = angle4 = 90 degree
                    if (angle5 > 160 and angle5 < 200) and (angle6 > 160 and angle6 < 200): # (angle5=130 and angle6=180) or (angle6=130 and angle5=180)
                        if (angle8 > 160 and angle8 < 200) and (angle7 > 160 and angle7 < 200): # angle7 = angle8 = 180 degree
                            label = 'T Pose'
                            color = (0, 255, 0)  # Green for T Pose

            # Define pose based on the angles
            if angle2 > 160 and angle2 < 200 and angle1 > 160 and angle1 < 200:  # angle1 = angle2 = 180 degree
                if angle4 > 160 and angle4 < 200 and angle3 > 160 and angle3 < 200: # angle3 = angle4 = 180 degree
                    if (angle5 > 110 and angle5 < 150) or (angle6 > 110 and angle6 < 150): # (angle5=130 and angle6=180) or (angle6=130 and angle5=180)
                        label = 'Tree Pose'
                        color = (0, 255, 0)  # Green for Tree Pose

            # Define pose based on the angles
            if angle2 > 160 and angle2 < 200 and angle1 > 160 and angle1 < 200:  # angle1 = angle2 = 180 degree
                if angle4 > 70 and angle4 < 110 and angle3 > 70 and angle3 < 110: # angle3 = angle4 = 90 degree
                    if (angle5 > 70 and angle5 < 110 and angle6 >120 and angle6 < 160) or  (angle6 > 70 and angle6 < 110 and angle5 >120 and angle5 < 160): #angle5=90 and angle6=140
                            if (angle8 > 160 and angle8 < 200 and angle7 > 70 and angle7 < 110) or (angle7 > 160 and angle7 < 200 and angle8 > 70 and angle8 < 110):  #angle7=90 and angle8=180
                                label = 'Warrior II Pose'
                                color = (255, 215, 0)  # Gold color for Warrior II Pose

            # Display all angles with their names on the image
            cv2.putText(frame, 'Shoulder-Elbow-Wrist L: {:.2f}, R: {:.2f}'.format(angle1, angle2), (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            cv2.putText(frame, 'Elbow-Shoulder-Hip R: {:.2f}, L: {:.2f}'.format(angle3, angle4), (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            cv2.putText(frame, 'Shoulder-Hip-Knee R: {:.2f}, L: {:.2f}'.format(angle5, angle6), (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            cv2.putText(frame, 'Hip-Knee-Ankle R: {:.2f}, L: {:.2f}'.format(angle7, angle8), (10, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            # Display the pose label
            cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error in classify_pose: {str(e)}")

    return frame


def main():
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame)

            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )
                frame = classify_pose(results.pose_landmarks.landmark, frame)

            cv2.imshow('Yoga Pose Classifier', frame)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
#实时调用摄像头检测 - 三个pose版本 - T,TREE,WARRIOR
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    a = np.array(a)  # Point A coordinates
    b = np.array(b)  # Point B coordinates (vertex)
    c = np.array(c)  # Point C coordinates
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return angle

def classify_pose(landmarks, frame):
    """Classify the yoga pose based on the angles of joints and display all angles."""
    label = 'Unknown Pose'
    color = (0, 0, 255)  # Default to red for unknown pose

    if landmarks:
        try:
            # Extract landmarks for the required joints
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Calculate angles
            angle1 = calculate_angle(left_shoulder, left_elbow, left_wrist)
            angle2 = calculate_angle(right_shoulder, right_elbow, right_wrist)
            angle3 = calculate_angle(right_elbow, right_shoulder, right_hip)
            angle4 = calculate_angle(left_elbow, left_shoulder, left_hip)
            angle5 = calculate_angle(right_shoulder, right_hip, right_knee)
            angle6 = calculate_angle(left_shoulder, left_hip, left_knee)
            angle7 = calculate_angle(right_hip, right_knee, right_ankle)
            angle8 = calculate_angle(left_hip, left_knee, left_ankle)

            # Define pose based on the angles
            if angle2 > 155 and angle2 < 195 and angle1 > 155 and angle1 < 195:
                if angle4 > 70 and angle4 < 110 and angle3 > 70 and angle3 < 110:
                    if angle8 > 165 and angle8 < 195 or angle7 > 165 and angle7 < 195:
                        if (angle8 > 80 and angle8 < 120) or (angle7 > 80 and angle7 < 120):
                            label = 'Warrior II Pose'
                            color = (255, 215, 0)  # Gold color for Warrior II Pose
                    if (angle8 > 160 and angle8 < 195) and (angle7 > 160 and angle7 < 195):
                        label = 'T Pose'
                        color = (0, 255, 0)  # Green for T Pose
            if (angle8 > 165 and angle8 < 195) or (angle7 > 165 and angle7 < 195):
                if (angle7 > 20 and angle7 < 45) or (angle8 > 20 and angle8 < 45):
                    label = 'Tree Pose'
                    color = (128, 0, 128)  # Purple for Tree Pose

            # Display all angles on the image
            cv2.putText(frame, f'Angles: {angle1:.2f}, {angle2:.2f}, {angle3:.2f}, {angle4:.2f}, {angle5:.2f}, {angle6:.2f}, {angle7:.2f}, {angle8:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            # Display the pose label
            cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error in classify_pose: {str(e)}")

    return frame


def main():
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame)

            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )
                frame = classify_pose(results.pose_landmarks.landmark, frame)

            cv2.imshow('Yoga Pose Image', frame)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [ ]:
#实时调用摄像头检测 - 三个pose版本 - DOG,TREE,WARRIOR
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    a = np.array(a)  # Point A coordinates
    b = np.array(b)  # Point B coordinates (vertex)
    c = np.array(c)  # Point C coordinates
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180:
        angle = 360 - angle
    return angle


def classify_pose(landmarks, frame):
    """Classify the yoga pose based on the angles of joints and display all angles."""
    label = 'Unknown Pose'
    color = (0, 0, 255)  # Default to red for unknown pose

    if landmarks:
        try:
            # Extract landmarks for the required joints
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                           landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Calculate angles
            angle1 = calculate_angle(left_shoulder, left_elbow, left_wrist)
            angle2 = calculate_angle(right_shoulder, right_elbow, right_wrist)
            angle3 = calculate_angle(right_elbow, right_shoulder, right_hip)
            angle4 = calculate_angle(left_elbow, left_shoulder, left_hip)
            angle5 = calculate_angle(right_shoulder, right_hip, right_knee)
            angle6 = calculate_angle(left_shoulder, left_hip, left_knee)
            angle7 = calculate_angle(right_hip, right_knee, right_ankle)
            angle8 = calculate_angle(left_hip, left_knee, left_ankle)

            # Define pose based on the angles
            if 160 <= angle1 <= 200 and 160 <= angle2 <= 200:
                label = 'Warrior II Pose'
                color = (255, 215, 0)  # Gold color for Warrior II Pose
            elif 45 <= angle1 <= 70 and 45 <= angle2 <= 70:
                label = 'Tree Pose'
                color = (128, 0, 128)  # Purple for Tree Pose
            elif 90 <= angle1 <= 120 and 90 <= angle2 <= 120:
                label = 'Dog Pose'
                color = (255, 0, 0)  # Red for Dog Pose

            # Display the pose label
            cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        
        except Exception as e:
            print(f"Error in classify_pose: {str(e)}")

    return frame



def main():
    cap = cv2.VideoCapture(0)
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                continue

            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame)

            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )
                frame = classify_pose(results.pose_landmarks.landmark, frame)

            cv2.imshow('Yoga Pose Image', frame)
            if cv2.waitKey(5) & 0xFF == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
